###Question: Calculate Gender-wise Employee Percentage

In [0]:
%sql
CREATE OR REPLACE TABLE employee (
    id INT,
    name STRING,
    gender STRING
);
INSERT INTO employee VALUES
(1, 'Alice', 'Female'),
(2, 'Bob', 'Male'),
(3, 'Carol', 'Female'),
(4, 'David', 'Male'),
(5, 'Eve', 'Female');


In [0]:
%sql
create or replace Temp view emp_view as
select * from employee

## Output

In [0]:
%sql
select 'Female' as gender,'60.00%' as percentage
Union
select 'Male' as gender,'40.00%' as percentage

In [0]:
%sql
Select * from employee;

In [0]:
%sql
with gender_date as(
SELECT gender,
sum(case when lower(gender) = lower('Female') then 1 else 0 end) female_count,
sum(case when lower(gender) = lower('male') then 1 else 0 end) male_count,
count(*) as total_count_grp
FROM employee
GROUP BY gender
),
total_count as (
SELECT sum(total_count_grp) as total_count
FROM gender_date
)
select gender,
CASE WHEN lower(gender) = lower('Female') THEN cast((female_count/total_count*100 ) as int)
WHEN lower(gender) = lower('male') THEN male_count/total_count*100
END as percentage
from gender_date,total_count



In [0]:
%sql
SELECT
    gender,
    ROUND(
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (),
        2
    ) AS percentage
FROM employee
GROUP BY gender;


#### Pyspark


In [0]:
spark.table('emp_view').display()

In [0]:
from pyspark.sql import functions as f
from pyspark.sql import Window as w
from pyspark.sql import types as t

In [0]:
win = w.partitionBy()
(spark.table('emp_view')
 .groupBy('gender').agg(f.count('*').alias('gender_count'))
 .withColumn('percentage',
             f.round(
                 f.col('gender_count') * 100.00 / f.sum('gender_count').over(win),
                 2
             )
             ).select('gender','percentage')
).display()


In [0]:
total_count = spark.table('emp_view').count()

result_df = (
    spark.table('emp_view')
    .groupBy("gender")
    .agg(
        F.round(F.count("*") * 100.0 / F.lit(total_count), 2)
        .alias("percentage")
    )
)

result_df.display()
